In [1]:
# Copyright 2020 The HuggingFace Team. All rights reserved.
# Copyright 2022 Vladislav Lialin
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [2]:
#! pip install datasets transformers wandb

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers

from tqdm.auto import tqdm

print(transformers.__version__)

4.17.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/question-answering).

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context.

![Widget inference representing the QA task](images/question_answering.png)

**Note:** This notebook finetunes models that answer question by taking a substring of a context, not by generating new text.

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [4]:
# Feel free to use other models instead of BERT-base-uncased. If you are limited in resources, you can try distilbert-base-uncased.
# YOUR CODE STARTS HERE
model_name = "bert-base-uncased"
# YOUR CODE ENDS HERE

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [5]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [6]:
datasets = load_dataset("squad_v2")

Reusing dataset squad_v2 (/home/bjeon/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [8]:
datasets["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5731a8d20fdd8d15006c6437,Indigenous_peoples_of_the_Americas,"However, since the 20th century, indigenous peoples in the Americas have been more vocal about the ways they wish to be referred to, pressing for the elimination of terms widely considered to be obsolete, inaccurate, or racist. During the latter half of the 20th century and the rise of the Indian rights movement, the United States government responded by proposing the use of the term ""Native American,"" to recognize the primacy of indigenous peoples' tenure in the nation, but this term was not fully accepted. Other naming conventions have been proposed and used, but none are accepted by all indigenous groups.","Who initially proposed using the term ""Native American"" to recognize the primacy of the native population?","{'text': ['United States government'], 'answer_start': [319]}"
1,572ef9b8c246551400ce4807,Antenna_(radio),"Typically an antenna consists of an arrangement of metallic conductors (elements), electrically connected (often through a transmission line) to the receiver or transmitter. An oscillating current of electrons forced through the antenna by a transmitter will create an oscillating magnetic field around the antenna elements, while the charge of the electrons also creates an oscillating electric field along the elements. These time-varying fields radiate away from the antenna into space as a moving transverse electromagnetic field wave. Conversely, during reception, the oscillating electric and magnetic fields of an incoming radio wave exert force on the electrons in the antenna elements, causing them to move back and forth, creating oscillating currents in the antenna.",During what process would electrons be vacillating in the antenna?,"{'text': ['reception'], 'answer_start': [559]}"
2,570962e2ed30961900e8404f,Copper,"Copper is synthesized in massive stars and is present in the Earth's crust at a concentration of about 50 parts per million (ppm), where it occurs as native copper or in minerals such as the copper sulfides chalcopyrite and chalcocite, the copper carbonates azurite and malachite, and the copper(I) oxide mineral cuprite. The largest mass of elemental copper discovered weighed 420 tonnes and was found in 1857 on the Keweenaw Peninsula in Michigan, US. Native copper is a polycrystal, with the largest described single crystal measuring 4.4×3.2×3.2 cm.",How large was the biggest find of copper?,"{'text': ['420 tonnes'], 'answer_start': [378]}"
3,5728166c4b864d190016444d,"St._John%27s,_Newfoundland_and_Labrador","St. John's is located along the coast of the Atlantic Ocean, on the northeast of the Avalon Peninsula in southeast Newfoundland. The city covers an area of 446.04 square kilometres (172.22 sq mi) and is the most easterly city in North America, excluding Greenland; it is 295 miles (475 km) closer to London, England than it is to Edmonton, Alberta. The city of St. John's is located at a distance by air of 3,636 kilometres (2,259 mi) from Lorient, France which lies on a nearly precisely identical latitude across the Atlantic on the French western coast. The city is the largest in the province and the second largest in the Atlantic Provinces after Halifax, Nova Scotia. Its downtown area lies to the west and north of St. John's Harbour, and the rest of the city expands from the downtown to the north, south, east and west.",What is the largest Atlantic Province city?,"{'text': ['Halifax'], 'answer_start': [652]}"
4,5a3ad5f03ff257001ab842ab,Flowering_plant,"Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specializ

## Preprocessing the training data

In this section we will learn some of the advanced features of the `Tokenizer` object.

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [13]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [14]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [15]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


Without any truncation, we get the following length for the input IDs:

In [16]:
len(tokenizer(example["question"], example["context"])["input_ids"])

518

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [17]:
# Task 2.1: tokenize the example found earlier and truncate it to max_length. Print out the length.
# Note that we never want to truncate the question (the first sequence),
# we only want to truncate the context (the second sequence).
# This can be achieved by providing a correct value to the `truncation` argument that defines the truncation strategy.
# Print out the length of the truncated example.
# YOUR CODE STARTS HERE
long_example_ids = tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"]
print(len(long_example_ids))
# YOUR CODE ENDS HERE

384


> Note that we never want to truncate the question, only the context, else the `only_second` truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with `return_overflowing_tokens=True` and by passing the stride:

In [18]:
# Task 2.2: tokenize the same example, but additionally
# * do striding
# * return overflowing tokens
# You can learn more about Tokenizer object here: https://huggingface.co/docs/transformers/main/en/main_classes/tokenizer
# Assign the output of the tokenizer to `tokenized_example`
# YOUR CODE STARTS HERE
tokenized_example = tokenizer(example["question"],
                              example["context"],
                              max_length=max_length,
                              truncation="only_second",
                              return_overflowing_tokens=True,
                              stride=doc_stride)
# YOUR CODE ENDS HERE

Now we don't have one list of `input_ids`, but several: 

In [19]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 275]

And if we decode them, we can see the overlap:

In [20]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] when were the rinbung princes overthrown? [SEP] in 1565, the powerful rinbung princes were overthrown by one of their own ministers, karma tseten who styled himself as the tsangpa, " the one of tsang ", and established his base of power at shigatse. the second successor of this first tsang king, karma phuntsok namgyal, took control of the whole of central tibet ( u - tsang ), reigning from 1611 – 1621. despite this, the leaders of lhasa still claimed their allegiance to the phagmodru as well as the gelug, while the u - tsang king allied with the karmapa. tensions rose between the nationalistic u - tsang ruler and the mongols who safeguarded their mongol dalai lama in lhasa. the fourth dalai lama refused to give an audience to the u - tsang king, which sparked a conflict as the latter began assaulting gelug monasteries. chen writes of the speculation over the fourth dalai lama's mysterious death and the plot of the u - tsang king to have him murdered for " cursing " him with illne

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [21]:
# Task 2.3: please carefully read the explanation why we need offset mapping above
# Now, ask the tokenizer to return you the offset mapping.
# Assign the output to `tokenized_example`
# YOUR CODE STARTS HERE
tokenized_example = tokenizer(example["question"],
                              example["context"],
                              max_length=max_length,
                              truncation="only_second",
                              return_overflowing_tokens=True,
                              return_offsets_mapping=True,
                              stride=doc_stride)
print(tokenized_example["offset_mapping"][0][:100])
# YOUR CODE ENDS HERE

[(0, 0), (0, 4), (5, 9), (10, 13), (14, 16), (16, 18), (18, 21), (22, 29), (30, 39), (39, 40), (40, 41), (0, 0), (0, 2), (3, 6), (6, 7), (7, 8), (9, 12), (13, 21), (22, 24), (24, 26), (26, 29), (30, 37), (38, 42), (43, 52), (52, 53), (54, 56), (57, 60), (61, 63), (64, 69), (70, 73), (74, 83), (83, 84), (85, 90), (91, 93), (93, 96), (96, 97), (98, 101), (102, 108), (109, 116), (117, 119), (120, 123), (124, 126), (126, 129), (129, 131), (131, 132), (133, 134), (134, 137), (138, 141), (142, 144), (145, 147), (147, 150), (150, 151), (151, 152), (153, 156), (157, 168), (169, 172), (173, 177), (178, 180), (181, 186), (187, 189), (190, 193), (193, 196), (196, 198), (198, 199), (200, 203), (204, 210), (211, 220), (221, 223), (224, 228), (229, 234), (235, 237), (237, 240), (241, 245), (245, 246), (247, 252), (253, 255), (255, 258), (258, 260), (260, 261), (262, 265), (265, 267), (267, 269), (269, 270), (271, 275), (276, 283), (284, 286), (287, 290), (291, 296), (297, 299), (300, 307), (308, 313

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [22]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

when When


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [23]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [24]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

13 14


And we can double check that it is indeed the theoretical answer:

In [25]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

1565
1565


Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [26]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        # This is how we will teach the model to reply "no answer exists" and we will be using this when working with the trained model.
        if len(answers["answer_start"]) == 0:
            # Task 2.4: this example is a no-answer example in the dataset
            # set the cls_index as the start and end position of the answer
            # (append it to tokenized_examples)
            # YOUR CODE STARTS HERE
            # (our implementation is 2 lines)
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
            # YOUR CODE ENDS HERE
        else:
            # Start/end character index of the answer in the text.
            # Task 2.5: set the start and end position of the answer to `start_char` and `end_char`
            # You can determine the end position using the start position and the length of the answer.
            # (our implementation is 2 lines)
            # YOUR CODE STARTS HERE
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # YOUR CODE ENDS HERE

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                # Task 2.5: this context does not contain the answer, so we treat it a no-answer example above
                # set the cls_index as the start and end position of the answer
                # (append it to tokenized_examples)
                # YOUR CODE STARTS HERE
                # (our implementation is 2 lines)
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
                # YOUR CODE ENDS HERE
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

                # Inline question 2: what does the code above do?
                # Why do we care about the condition `offsets[token_start_index][0] <= start_char`?
                # The answer "because this determines the start position of the answer in the context"
                # is **not** detailed enough. Elaborate it.
                # YOUR ANSWER HERE: To advance with the start token pointer till the start_char is reached

    return tokenized_examples

**Inline question 2:** what does this code do (it is from the `prepare_train_features` funciton above)?

```python
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
            token_start_index += 1
        tokenized_examples["start_positions"].append(token_start_index - 1)
        while offsets[token_end_index][1] >= end_char:
            token_end_index -= 1
        tokenized_examples["end_positions"].append(token_end_index + 1)
```

Why do we care about the condition `offsets[token_start_index][0] <= start_char`?

The answer "because this determines the start position of the answer in the context"
is **not** detailed enough. Elaborate it.

**YOUR ANSWER HERE:**

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [27]:
features = prepare_train_features(datasets['train'][:5])

**Inline question 1**: Look at the features object and describe what fields it has and what it the purpose of each of them

**YOUR ANSWER HERE:** features object has input_ids, token_type_ids, attention_mask, start_positions, end_positions columns fields. The purpose of each of them is:

input_ids: Indices of input sequence tokens in the vocabulary. 
attention_mask: Mask to avoid performing attention on padding token indices. 
token_type_ids: Segment token indices to indicate first and second portions of the inputs. 
start_positions: labels for position (index) of the start of the labelled span for computing the token classification loss. end_positions: Labels for position (index) of the end of the labelled span for computing the token classification loss. 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [28]:
# Task 2.6: apply prepare_train_features to every example in the dataset using map
# Provide `remove_columns=datasets["train"].column_names`
# We need to do that, because `prepare_train_features` cnanges the
# number of examples in the dataset and old columns do not align with the new ones.
# YOUR CODE STARTS HERE
tokenized_datasets = datasets.map(prepare_train_features, load_from_cache_file=False, remove_columns=datasets["train"].column_names, batched=True)
# YOUR CODE ENDS HERE

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready for training, we can download the pretrained model and fine-tune it. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us:

> Note: Transformers also provides `AutoModelForQuestionAnswering`, but we want you to implement the model by yourself starting from a regular `AutoModel`.

In [29]:
from transformers import AutoModel, TrainingArguments, Trainer

bert_model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

We have discussed the basic idea of how to use BERT for extractive question answering in the class. Feel free to lookup [the slides](https://docs.google.com/presentation/d/1SpzwnQoUrqzfvDJfwCJWT_KAh7kLU0uT9c-sbPZTXWw/edit?usp=sharing) and the lecture recording. Additionally, you might find the [original BERT paper](https://arxiv.org/abs/1810.04805) useful. It is well-written and should not be too hard to read.

Now, implement an nn.Module that accepts the processed data and predicts the start index of the the answer and the end index of the answer. Note that the model returns hiddens of all `input_ids` including the question. You would want to ignore the `input_ids` of the question when computing the loss and predicting the answer. Also note how the case of no-answer is handled.

In [30]:
# Task 2.7: Implement BertForQuestionAnswering model
# YOUR CODE STARTS HERE
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(model_name)
# YOUR CODE ENDS HERE

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

Next step is batching. Create Dataloaders that will batch your data.
Feel free to use the HuggingFace's `default_data_collator` or to write your own. Make sure your dataloader works before proceeding to the next step.

In [31]:
from transformers import default_data_collator

# Task 2.8: Create collator, train_dataloader, and valid_dataloader
# YOUR CODE STARTS HERE
collator = default_data_collator
train_dataloader = torch.utils.data.DataLoader(tokenized_datasets["train"], collate_fn=collator)
valid_dataloader = torch.utils.data.DataLoader(tokenized_datasets["validation"], collate_fn=collator)
# YOUR CODE ENDS HERE
next(iter(train_dataloader))

{'input_ids': tensor([[  101,  2043,  2106, 20773,  2707,  3352,  2759,  1029,   102, 20773,
          21025, 19358, 22815,  1011,  5708,  1006,  1013, 12170, 23432, 29715,
           3501, 29678, 12325, 29685,  1013, 10506,  1011, 10930,  2078,  1011,
           2360,  1007,  1006,  2141,  2244,  1018,  1010,  3261,  1007,  2003,
           2019,  2137,  3220,  1010,  6009,  1010,  2501,  3135,  1998,  3883,
           1012,  2141,  1998,  2992,  1999,  5395,  1010,  3146,  1010,  2016,
           2864,  1999,  2536,  4823,  1998,  5613,  6479,  2004,  1037,  2775,
           1010,  1998,  3123,  2000,  4476,  1999,  1996,  2397,  4134,  2004,
           2599,  3220,  1997,  1054,  1004,  1038,  2611,  1011,  2177, 10461,
           1005,  1055,  2775,  1012,  3266,  2011,  2014,  2269,  1010, 25436,
          22815,  1010,  1996,  2177,  2150,  2028,  1997,  1996,  2088,  1005,
           1055,  2190,  1011,  4855,  2611,  2967,  1997,  2035,  2051,  1012,
           2037, 14221,  23

In [32]:
# Feel free to change the parameters below
# YOUR CODE STARTS HERE
learning_rate = 2e-5
batch_size = 16
num_train_epochs = 1
weight_decay = 0.01
# YOUR CODE ENDS HERE

Now, implement the training loop. Do **not** use HuggingFace's `Trainer` class. Implement it from scratch (it is probably easier to do than to learn how to apply Trainer for this task anyway).

Note that correctly evaluating a SQuAD model is quite complicated (look at the next task). For this reason, during training you can track some simpler metrics on token-id level (not word-level), such as "accuracy of answer start", "accuracy of answer end" and "exact match" or even just validation loss.

In [33]:
import wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
run = wandb.init(project=f"bert_QA_SQuAD")

# Feel free to modify the code below
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

bert_model = AutoModel.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Task 2.9: Training loop
# YOUR CODE STARTS HERE
global_step = 0
for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions = start_positions,
            end_positions = end_positions
        )

        loss = F.cross_entropy(
            ignore_index=tokenizer.pad_token_id,
        )

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        global_step += 1

        wandb.log(
            {
                "train_loss": loss,
                "learning_rate": optimizer.param_groups[0]["lr"],
                "epoch": epoch,
            },
            step=global_step,
        )
# YOUR CODE ENDS HERE

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.pr

## Predicting answers with the trained model

In [ ]:
# Task 2.10: predict `start_logits` and `end_logits` of the first example in the validation set by your trained model
# YOUR CODE STARTS HERE
start_logits, end_logits = output.start_logits.shape, output.end_logits.shape
# YOUR CODE ENDS HERE

We have one logit for each feature and each token. The most obvious thing to predict an answer for each featyre is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

```python
    output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)
    
    (tensor([ 46,  57,  78,  43, 118,  15,  72,  35,  15,  34,  73,  41,  80,  91,
             156,  35], device='cuda:0'),
     tensor([ 47,  58,  81,  55, 118, 110,  75,  37, 110,  36,  76,  53,  83,  94,
             158,  35], device='cuda:0'))
```

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.

To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [ ]:
n_best_size = 20

start_logits = start_logits[0].cpu().numpy()
end_logits = end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = start_logits[0].cpu().numpy()
end_logits = end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

We can compare to the actual ground-truth answer:

In [ ]:
datasets["validation"][0]["answers"]

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer. The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, predictions)

Then we can load the metric from the datasets library.

In [ ]:
metric = load_metric("squad_v2")

Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

We expect your model to achieve EM of 0.7 or more.

In [ ]:
# Task 2.11: The last task is to find an examle where model predicts a correct answer and another one where it doesn't.
# Feel free to use more than one jupyter cell.
# YOUR CODE STARTS HERE
trainer.push_to_hub()
# YOUR CODE ENDS HERE